In [1]:
import pandas as pd
import numpy as np

Informação interessante para mim:

1. Nome do filme
2. Data que assistiw
3. Se reassisti
4. Nota
5. Ano do filme?

Gostaria de ter o diretor do filme.

### Questions

* Is every rewatched movie marked as rewatch (cleaning)
* How does my ratings compare to the movies average rating (enrichment)
    * I need to get the movies avaerage rating information from somewhere
* How does letterboxd avaliations compare to IMDB? (EDA, maybe that needs to be done elsewhere in another notebook)

## Reading

In [2]:
PROFILE_DATA_PATH = "profile_data"
DIARY_PATH = f"{PROFILE_DATA_PATH}/diary.csv"
RATINGS_PATH = f"{PROFILE_DATA_PATH}/ratings.csv"
REVIEWS_PATH = f"{PROFILE_DATA_PATH}/reviews.csv"
WATCHED_PATH = f"{PROFILE_DATA_PATH}/watched.csv"
WATCHLIST_PATH = f"{PROFILE_DATA_PATH}/watchlist.csv"

In [3]:
diary     = pd.read_csv(DIARY_PATH    )
ratings   = pd.read_csv(RATINGS_PATH  )

reviews   = pd.read_csv(REVIEWS_PATH  )
watched   = pd.read_csv(WATCHED_PATH  )
watchlist = pd.read_csv(WATCHLIST_PATH)

In [4]:
dfs = {"diary": diary, "ratings": ratings, "reviews": reviews, "watched": watched, "watchlist": watchlist}

In [5]:
diary

,Date,Name,Year,Letterboxd URI,Rating,Rewatch,Tags,Watched Date
0,2021-08-01,Avengers: Endgame,2019,https://boxd.it/22iXvd,4.0,NaN,NaN,2019-04-13
1,2021-08-01,The Girl Who Leapt Through Time,2006,https://boxd.it/22iXgn,4.0,NaN,NaN,2020-12-30
2,2021-08-01,Soul,2020,https://boxd.it/22iWYD,4.5,NaN,NaN,2021-01-03
3,2021-08-01,Saving Private Ryan,1998,https://boxd.it/22iWSF,4.5,NaN,NaN,2021-01-18
4,2021-08-01,Tenet,2020,https://boxd.it/22iWJj,4.0,NaN,NaN,2021-02-26
...,...,...,...,...,...,...,...,...
195,2023-05-18,Guardians of the Galaxy Vol. 3,2023,https://boxd.it/4gamiR,2.0,NaN,muito bom deveria ter esperado sair no disney+,2023-05-17
196,2023-05-21,Guardians of the Galaxy Vol. 2,2017,https://boxd.it/4gPDGX,4.0,NaN,NaN,2023-05-20
197,2023-05-22,Avengers: Infinity War,2018,https://boxd.it/4h7xBj,4.5,Yes,NaN,2023-05-20
198,2023-05-22,Broker,2022,https://boxd.it/4h7wib,4.0,NaN,NaN,2023-05-21


Columns description:

* Date: date in which the record was created at letterbox
* Name: name of the movie
* Year: movie's release year
* Ltterboxd URI: movie URI
* Rating: my provided rating
* Rewatch: if the record is a rewatch or not
* Tags: user inputed tags
* Watched date: date that the movie was watched

In [6]:
for name, df in dfs.items():
    for col in df.columns:
        s = df[col].isna().sum()
        if(s > 0):
            print(f"Found {s} NaN values in column {col} in {name}.")

Found 175 NaN values in column Rewatch in diary.
Found 133 NaN values in column Tags in diary.
Found 97 NaN values in column Rewatch in reviews.
Found 69 NaN values in column Tags in reviews.
Found 1 NaN values in column Year in watchlist.


### building the dataset

- [X] Remove Nan values
- [X] Map rewatch number of times the movie was watched
- [X] Drop unused columns
- [X] Remove duplicates
- [X] Check which movies are on rating that are not in diary
- [X] Check how movie rating changes after a rewatch (if there is a reting prior)
    - it barely changes

In [7]:
movies_ratings = set(ratings['Name'])
movies_diary= set(diary['Name'])

only_in_ratings_movies = movies_ratings - movies_ratings.intersection(movies_diary)

In [8]:
only_in_ratings_df = ratings[ratings['Name'].isin(only_in_ratings_movies)]

In [9]:
only_in_ratings_df

,Date,Name,Year,Letterboxd URI,Rating
18,2021-08-01,Jiro Dreams of Sushi,2011,https://boxd.it/2ONU,4.5
19,2021-08-01,Marriage Story,2019,https://boxd.it/hJAw,4.5
20,2021-08-01,The Meyerowitz Stories (New and Selected),2017,https://boxd.it/dSp0,4.0
21,2021-08-01,The Squid and the Whale,2005,https://boxd.it/1VAG,4.0
22,2021-08-01,Your Name.,2016,https://boxd.it/cUqs,5.0
...,...,...,...,...,...
126,2021-08-05,For a Few Dollars More,1965,https://boxd.it/29eu,4.5
127,2021-08-06,Rushmore,1998,https://boxd.it/1TAO,3.0
129,2021-08-06,Dirty Harry,1971,https://boxd.it/299E,4.0
136,2021-08-15,BlacKkKlansman,2018,https://boxd.it/hxL8,4.0


In [10]:
df = pd.concat([diary, only_in_ratings_df])

In [11]:
df

,Date,Name,Year,Letterboxd URI,Rating,Rewatch,Tags,Watched Date
0,2021-08-01,Avengers: Endgame,2019,https://boxd.it/22iXvd,4.0,NaN,NaN,2019-04-13
1,2021-08-01,The Girl Who Leapt Through Time,2006,https://boxd.it/22iXgn,4.0,NaN,NaN,2020-12-30
2,2021-08-01,Soul,2020,https://boxd.it/22iWYD,4.5,NaN,NaN,2021-01-03
3,2021-08-01,Saving Private Ryan,1998,https://boxd.it/22iWSF,4.5,NaN,NaN,2021-01-18
4,2021-08-01,Tenet,2020,https://boxd.it/22iWJj,4.0,NaN,NaN,2021-02-26
...,...,...,...,...,...,...,...,...
126,2021-08-05,For a Few Dollars More,1965,https://boxd.it/29eu,4.5,NaN,NaN,NaN
127,2021-08-06,Rushmore,1998,https://boxd.it/1TAO,3.0,NaN,NaN,NaN
129,2021-08-06,Dirty Harry,1971,https://boxd.it/299E,4.0,NaN,NaN,NaN
136,2021-08-15,BlacKkKlansman,2018,https://boxd.it/hxL8,4.0,NaN,NaN,NaN


### Cleaning and formatting

In [12]:
df['Rewatch'] = df['Rewatch'].map({'Yes': 1, np.nan: 0})
df['movie_id'] = df['Letterboxd URI'].map(lambda s: s.split('/')[-1])
df['movie_id'] = df['movie_id'].astype(str)
diary = diary.drop(columns=['Tags', 'Letterboxd URI', 'Date'])

watchlist = watchlist.dropna()

In [13]:
rewatches = df.groupby('Name').size().reset_index(name='count')

In [14]:
df = df.groupby('Name')\
     .agg(\
          {'Year': min,\
           'Rating': np.average,\
           'Rewatch': lambda x: x.sum() + 1,\
           'Watched Date': max,\
           'movie_id': min})\
      .rename(
          columns={'Rating': 'Average Rating', 'Rewatch': 'Times watched', 'Watched Date': 'latest Date'}
      ).reset_index()

In [15]:
df

,Name,Year,Average Rating,Times watched,latest Date,movie_id
0,(500) Days of Summer,2009,4.5,2,2022-04-02,23pe3V
1,10 Things I Hate About You,1999,3.5,1,2023-04-01,43Uvux
2,12 Angry Men,1957,5.0,1,2021-08-02,22yBiD
3,1917,2019,4.0,1,NaN,jj4y
4,2001: A Space Odyssey,1968,4.5,2,2021-08-28,26hLtX
...,...,...,...,...,...,...
285,Yesterday,2019,3.0,1,2021-04-03,22iWEX
286,Your Name.,2016,5.0,1,NaN,cUqs
287,Zodiac,2007,4.5,1,NaN,27MO
288,mother!,2017,4.5,1,2021-12-28,2plM9N


### EDA

Some questions:

* How distributed is my rating
* How differently rated are movies that I have rewatched
* Do i have a bias towards a certain decade or year?
* What day of the week have I watched more movies?
* What's the month in which have I watched more movies?
* What's the shortest i have waited among rewatches? What's the longest?